# **CLOUD SIDE**
GOOGLE MAPS API?!

In [ ]:
!pip install python-google-places

  Created wheel for python-google-places: filename=python_google_places-1.4.2-py3-none-any.whl size=13606 sha256=98fda1349e1e3f9de4783357af9a52fe30a80176fc9a2d467d26f2d870bdeb23
  Stored in directory: /root/.cache/pip/wheels/e6/1b/c7/bcaf3c3822813b8b3d621e1b853a25db9788d9eb00e702af4e
Successfully built python-google-places


In [ ]:
import numpy as np
Max_ERs= 3
#assign a unique code to each ER room! the third number is the average service time (time spent at the doctor's clinic)
# means that the first Queue in the list is for Carelton's ER
ERsLookup = {
  'Carelton':0,
  'CHEO':1, 
  'Queens':2}
#in minutes
AvgServiceTimeLookup = {
  0:20,
  1:30,
  2:10
  }
#assumption here is that only 3 ERs are available
class CloudWaitRooms:
    
    WaitRooms =[[]] * 3
    
    def Addpatient(self, queueindex, patientid, patientTriagescore):
        self.WaitRooms[queueindex].append((patientTriagescore,patientid))
        self.WaitRooms[queueindex].sort() #set patient where they should be in queue, remember python executes stable sorting for duplicate values so the exsting values wont be affected !
        #plan B sorted_multi_list = sorted(self.WaitRooms[0], key=lambda x: x[0])
        return "Patient added successfully"
        ##################CHANGE THIS , THE PATIENT MUST BE ADDED AFTER THE PATIENT / SAME SCORE#################
    
    def ExpectedWaitTime(self,queueindex, patientid,triagescore ):
        #should run on a scheduled bases to keep patients updated
        #get the average service time of the selected ER
        AvgServiceTime=AvgServiceTimeLookup.get(queueindex)
        index = self.WaitRooms[queueindex].index((triagescore,patientid))
        return AvgServiceTime * (index+1)
        

    def RemovePatient(self, queueindex, patientid,triagescore):
        self.WaitRooms[queueindex].remove((triagescore,patientid))
        return 'Patient removed from queue successfully'
    
        
    def SelectBestER(self, patientid,patientTriagescore):
 
        API_KEY = 'Your_API_Key'
        google_places = GooglePlaces(API_KEY)
        query_result = google_places.nearby_search(
		    lat_lng ={'lat': 28.4089, 'lng': 77.3178},
		    radius = 5000,
	      types =[types.TYPE_HOSPITAL]) 
        if query_result.has_attributions:
	         print (query_result.html_attributions)
        #here's the list of all potential ERs, need to score them some how, then see which one offers minmum wait time and then return the best fit
        #CHANGE THIS CZ THE APPENDED ITEMS  WILL GO AFTER THE ZERO COLUMNS!:(
        WaiTimeScores= np.zeros((Max_ERs, 2))
        PlaceScores=np.zeros(Max_ERs)
        FinalScores=np.zeros(Max_ERs)
        for place in query_result.places:
           PlaceScores.append(queueindex)
	         #get the ER Queue index
           queueindex = ERsLookup.get(place.name)
           PlaceScores.append(queueindex)
           TempQ= self.WaitRooms[queueindex]
           TempQ[queueindex].append((patientTriagescore,patientid))
           TempQ[queueindex].sort()
           AvgServiceTime=AvgServiceTimeLookup.get(queueindex)
           index = TempQ[queueindex].index((patientTriagescore,patientid))
           Waitime = AvgServiceTime * (index+1)
           WaiTimeScores = np.append(WaiTimeScores, [[Waitime,queueindex]], axis=0)#each ER and its expected wait time 
           PlaceScores= np.append(PlaceScores, [[queueindex]], axis=0) #assumption is that Google returns places sorted by distance, if else then will need to add both queue and distance and then sort
        WaiTimeScores = WaiTimeScores[np.argsort(WaiTimeScores[:, 0])] #sorts ascending by dafault, so less wait time will have lower index ,IMP because the index will take the lead in calculating the score!
        min= 0
        max= len(WaiTimeScores) 
        #cant normalise since we dont have the max and min so better to do it based on indicies
        for idx, val in enumerate(WaiTimeScores):
            WaiTimeScores[idx][0] = round((idx - min)/(max-min), 2)
        for idx, val in enumerate(PlaceScores):
            PlaceScores[idx] = round((idx - min)/(max-min), 2)
        #location scores 
        alpha=.5
        for idx, val in enumerate(WaiTimeScores):
            FinalScores= np.append(FinalScores,(alpha * val + (1-alpha) * PlaceScores[idx]))
        #less score is better    
        return np.amin(FinalScores)
         
	         #print("Latitude", place.geo_location['lat'])
	         #print("Longitude", place.geo_location['lng'])
        
        

In [ ]:
#########REMINDER CHAnge THE ZEROS INITIALISE BECAUSE ITS TAKING EXTRA RECORDS
######### change the assign score line , need to map based on QUEUE ID!!!
######### need to return the best Queue id  not score
CloudWaitRoomsobject = CloudWaitRooms()
CloudWaitRoomsobject.Addpatient(0,2,2)
CloudWaitRoomsobject.Addpatient(0,22,2)
CloudWaitRoomsobject.ExpectedWaitTime(0,22,2)
#CloudWaitRooms.RemovePatient(0,3)
#CloudWaitRooms.SelectBestER(3,4) #wont work cz no google subscription


300